In [1]:

import numpy as np
import angular_momentum
import wigner_d
import cg
import math
import random
from wigner_d import WignerD
from hydrogen import hydrogen_matrix_element
from cg import ClebschGordanCoefficients
from IPython.display import display


# Matrix elements of an operator in the basis of the hydrogen atom energy eigenstates
Example: 
$$
\langle 2, 1, 0 | z | 1, 0, 0 \rangle = \frac{128\sqrt{2}}{243} a_0 \approx 0.744936 a_0
$$

In [2]:
# expressed in the natural unit of hydrogen atom (a0=1)
hydrogen_matrix_element(
    2,1,0,
    1,0,0,
    op = lambda r, theta, phi: r * np.cos(theta),  # z = r cos(theta)
    a = 1
)

0.7449355390278037

# Angular momentum operators
The angular momentum operators $J_x^{(j)}$, $J_y^{(j)}$, $J_z^{(j)}$, $J_+^{(j)}$, and $J_-^{(j)}$ in the bases $|j, m \rangle$ given a $j \in \{ n / 2 \mid n \in \mathbb{N} \}$. We will use the natural unit such that $\hbar = 1$.

In [3]:
j = 1/2
angular_momentum.J_x(j), angular_momentum.J_y(j), angular_momentum.J_z(j)


(array([[0. , 0.5],
        [0.5, 0. ]]),
 array([[0.+0.j , 0.+0.5j],
        [0.-0.5j, 0.+0.j ]]),
 array([[1., 0.],
        [0., 1.]]))

check that $\left( j_y^{(j=1)} \right)^3 = J_y^{(j=1)}$


In [4]:
j = 1
np.linalg.matrix_power(angular_momentum.J_y(j), 3) - angular_momentum.J_y(j)  # should be zero up to machine precision


array([[0.+0.00000000e+00j, 0.+1.11022302e-16j, 0.+0.00000000e+00j],
       [0.-1.11022302e-16j, 0.+0.00000000e+00j, 0.+1.11022302e-16j],
       [0.+0.00000000e+00j, 0.-1.11022302e-16j, 0.+0.00000000e+00j]])

In [5]:
angular_momentum.J_plus(1), angular_momentum.J_minus(1)

(array([[0.        , 0.        , 0.        ],
        [1.41421356, 0.        , 0.        ],
        [0.        , 1.41421356, 0.        ]]),
 array([[0.        , 1.41421356, 0.        ],
        [0.        , 0.        , 1.41421356],
        [0.        , 0.        , 0.        ]]))

# Wigner D matrices

In [6]:
# spin-1/2
# gives -I after a full rotation
j = 1 / 2
beta = 2 * np.pi
wigner_d.wigner_d(j, beta).round(12)


array([[-1.,  0.],
       [-0., -1.]])

In [7]:
j = 1
beta = 2 * np.pi
wigner_d.wigner_d(j, beta).round(12)


array([[ 1., -0.,  0.],
       [ 0.,  1., -0.],
       [ 0.,  0.,  1.]])

In [8]:
# block diagonal form of wigner d up to j = 3/2 at beta = pi/4
wigner_d.wigner_d_block_diag(jmax=3/2, beta=np.pi/4)


array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.92387953,  0.38268343,  0.        ,  0.        ,
         0.        ],
       [ 0.        , -0.38268343,  0.92387953,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.85355339,  0.5       ,
         0.14644661],
       [ 0.        ,  0.        ,  0.        , -0.5       ,  0.70710678,
         0.5       ],
       [ 0.        ,  0.        ,  0.        ,  0.14644661, -0.5       ,
         0.85355339]])

In [9]:
j = 1
alpha = random.uniform(0, 2 * math.pi)
beta = random.uniform(0, math.pi)
gamma = random.uniform(0, 2 * math.pi)
d = wigner_d.wigner_d(j, beta)
# given a d, we can calculate the D matrix (and vice versa using wigner_d.wigner_d_from_D))
wigner_d.wigner_D_from_d(d, alpha, gamma) - wigner_d.wigner_D(j, alpha, beta, gamma)


array([[0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j]])

In [10]:
alpha = np.random.rand(1) * 2 * np.pi
beta = np.random.rand(1) * np.pi
gamma = np.random.rand(1) * 2 * np.pi
wd = WignerD(jmax=2, alpha=alpha, beta=beta, gamma=gamma)
wd

WignerDDict(jmax=2, alpha=[3.93051752], beta=[3.0425003], gamma=[3.51596178])

In [11]:
# update jmax and stores wigner matrices up to jmax
wd.set_jmax(3)
wd


WignerDDict(jmax=3, alpha=[3.93051752], beta=[3.0425003], gamma=[3.51596178])

In [12]:
# look up the wigner D matrix with j = 1
wd.wigner_D(1)

array([[ 9.72093570e-04+0.00225196j, -4.92903763e-02-0.04963927j,
         9.13050303e-01+0.40179538j],
       [ 6.51091242e-02+0.02558125j, -9.95094369e-01+0.j        ,
        -6.51091242e-02+0.02558125j],
       [ 9.13050303e-01-0.40179538j,  4.92903763e-02-0.04963927j,
         9.72093570e-04-0.00225196j]])

In [13]:
# block diagonal wigner d matrix up to j = 3/2
wd.get_d_block(jmax=3/2)

array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  4.95259084e-02,  9.98772839e-01,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00, -9.98772839e-01,  4.95259084e-02,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         2.45281560e-03,  6.99542607e-02,  9.97547184e-01,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -6.99542607e-02, -9.95094369e-01,  6.99542607e-02,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.

In [14]:
# set a new angles
alpha = np.random.rand(1) * 2 * np.pi
beta = np.random.rand(1) * np.pi
gamma = np.random.rand(1) * 2 * np.pi

wd.set_alpha(alpha)
wd.set_beta(beta)
wd.set_gamma(gamma)


In [15]:
# a dictionary of all stored little d matrices
wd.d

{0.0: array([[1.]]),
 0.5: array([[ 0.04122332,  0.99914996],
        [-0.99914996,  0.04122332]]),
 1.0: array([[ 0.00169936,  0.05824902,  0.99830064],
        [-0.05824902, -0.99660128,  0.05824902],
        [ 0.99830064, -0.05824902,  0.00169936]]),
 1.5: array([[ 7.00533587e-05,  2.94087982e-03,  7.12795523e-02,
          9.97452040e-01],
        [-2.94087982e-03, -8.22364841e-02, -9.94056204e-01,
          7.12795523e-02],
        [ 7.12795523e-02,  9.94056204e-01, -8.22364841e-02,
          2.94087982e-03],
        [-9.97452040e-01,  7.12795523e-02, -2.94087982e-03,
          7.00533587e-05]]),
 2.0: array([[ 2.88783217e-06,  1.39987621e-04,  4.15549677e-03,
          8.22365734e-02,  9.96604163e-01],
        [-1.39987621e-04, -5.08653552e-03, -1.00547372e-01,
         -9.91514740e-01,  8.22365734e-02],
        [ 4.15549677e-03,  1.00547372e-01,  9.89821153e-01,
         -1.00547372e-01,  4.15549677e-03],
        [-8.22365734e-02, -9.91514740e-01,  1.00547372e-01,
         -5.08

# Clebsch-Gordan coefficients
The CG coefficient $\langle j_1, j_2; m_1, m_2 | j_1, j_2; j, m \rangle$

In [16]:
# compute a single Clebsch-Gordan coefficient
cg.cg_coefficient(
    j1=1/2, j2=1/2,
    m=0, j=1,
    m1=-1/2, m2=1/2
)

0.7071067811865476

In [17]:
# display a table of Clebsch-Gordan coefficients given j1, j2, and m
# should be empty: https://en.wikipedia.org/wiki/Table_of_Clebsch%E2%80%93Gordan_coefficients
cg.cg_table(j1=1, j2=0, m=1/2)


j,1.0
"(m1, m2)",


In [18]:
# https://en.wikipedia.org/wiki/Table_of_Clebsch%E2%80%93Gordan_coefficients
cg.cg_table(j1=1, j2=1/2, m=1/2)

j,0.5,1.5
"(m1, m2)",,
"(0.0, 0.5)",-0.577350,0.816497
"(1.0, -0.5)",0.816497,0.577350


In [19]:
# CG table with all possible m's
# returns a dict of pd.DataFrame
_ = cg.cg_tables_all_m(1/2, 1/2, display_tables=True)

j1 = 0.5, j2 = 0.5
m = -1.0:


j,1.0
"(m1, m2)",
"(-0.5, -0.5)",1.0


m = 0.0:


j,0.0,1.0
"(m1, m2)",,
"(-0.5, 0.5)",-0.707107,0.707107
"(0.5, -0.5)",0.707107,0.707107


m = 1.0:


j,1.0
"(m1, m2)",
"(0.5, 0.5)",1.0


In [20]:
# stores all CG coefficients up to jmax=2
cg_table = ClebschGordanCoefficients(jmax=2)
# get the pd.DataFrame for the CG coefficients for j1 = j2 = 1/2 and m = 1
cg_table[0.5, 0.5, 1]


j,1.0
"(m1, m2)",
"(0.5, 0.5)",1.0


In [21]:
cg_table


ClebschGordanCoefficients(jmax=2)

In [22]:
cg_table[5/2, 1]  # out of range -> update jmax to 5/2 automatically
cg_table

ClebschGordanCoefficients(jmax=2.5)

In [23]:
# displays the computed CG table with j1 = j2 = 1
for t in cg_table[1, 1].items():
    print(f'm = {t[0]}:')
    display(t[1])


m = -2.0:


j,2.0
"(m1, m2)",
"(-1.0, -1.0)",1.0


m = -1.0:


j,1.0,2.0
"(m1, m2)",,
"(-1.0, 0.0)",-0.707107,0.707107
"(0.0, -1.0)",0.707107,0.707107


m = 0.0:


j,0.0,1.0,2.0
"(m1, m2)",,,
"(-1.0, 1.0)",0.57735,-0.707107,0.408248
"(0.0, 0.0)",-0.57735,0.000000,0.816497
"(1.0, -1.0)",0.57735,0.707107,0.408248


m = 1.0:


j,1.0,2.0
"(m1, m2)",,
"(0.0, 1.0)",-0.707107,0.707107
"(1.0, 0.0)",0.707107,0.707107


m = 2.0:


j,2.0
"(m1, m2)",
"(1.0, 1.0)",1.0


In [24]:
# displays the computed CG table of with j1=3/2 and j2=1/2 with all possible m
# returns a dictionary of CG tables (pd.DataFrame)
_ = cg.cg_tables_all_m(j1=3/2, j2=1/2, display_tables=True)


j1 = 1.5, j2 = 0.5
m = -2.0:


j,2.0
"(m1, m2)",
"(-1.5, -0.5)",1.0


m = -1.0:


j,1.0,2.0
"(m1, m2)",,
"(-1.5, 0.5)",-0.866025,0.500000
"(-0.5, -0.5)",0.500000,0.866025


m = 0.0:


j,0.0,1.0,2.0
"(m1, m2)",,,
"(-0.5, 0.5)",0,-0.707107,0.707107
"(0.5, -0.5)",0,0.707107,0.707107


m = 1.0:


j,1.0,2.0
"(m1, m2)",,
"(0.5, 0.5)",-0.500000,0.866025
"(1.5, -0.5)",0.866025,0.500000


m = 2.0:


j,2.0
"(m1, m2)",
"(1.5, 0.5)",1.0
